In [1]:
import os

from matplotlib import pyplot as plt
from PIL import Image
import cv2
import numpy as np

import sqlite3

from utils import read_array, find_ground_plane, quaternion_rotation_matrix, calculate_distances

In [2]:
with open("./0/points3D.txt", "r") as f:
    temp_pts = f.readlines()[3:]
pts3d = np.array([x.split(" ")[1:4] for x in temp_pts], dtype=np.float32)
rgb3d = np.array([x.split(" ")[4:7] for x in temp_pts], dtype=np.float32)/255

with open("./0/images.txt", "r") as f:
    temp_images = f.readlines()[4:][::2]
img_ids = np.array([x.split(" ")[0] for x in temp_images], dtype=np.int16)
poses = np.array([x.split(" ")[1:8] for x in temp_images], dtype=np.float32)

with open("./0/cameras.txt", "r") as f:
    temp_cameras = f.readlines()[-1]
cam_params = np.array(temp_cameras.split(" ")[-4:], dtype=np.float32)

con = sqlite3.connect("./database.db")
cur = con.cursor()
img_dict = dict()
for row in cur.execute("SELECT * FROM IMAGES"):
    img_dict[row[0]] = row[1]
con.close()

In [3]:
# getting matrices
def get_extrinsic_matrix(pose):
    R = quaternion_rotation_matrix(pose[:4])
    T = pose[-3:]

    extrinsic = np.zeros((4,4))
    extrinsic[:3,:3] = R
    extrinsic[:3, -1] = T
    extrinsic[3,3] = 1

    return extrinsic

def get_K_matrix(cam_params):
    K = np.zeros((3,4))
    K[(0,1),(0,1)] = cam_params[0]
    K[:2, 2] = cam_params[1:3]
    K[2,2] = 1

    return K

def get_K_inv(K):
    kinv = np.zeros((4,3))
    f = K[0,0]
    kinv[(0,1),(0,1)] = 1/f
    kinv[2,2] = 1
    kinv[:2, 2] = -K[:2,2] / f
    return kinv
    

# getting view
def get_camera_view(points3d, extrinsic):
    camera_points = np.pad(points3d, pad_width=((0,0), (0,1)), mode="constant", constant_values=1) @ extrinsic.T
    return camera_points

def get_image_view(camera_points, intrinsic):
    pixel_points = camera_points @ intrinsic.T
    pixel_points = pixel_points / pixel_points[:,2:3]
    return pixel_points

def get_in_view_points(pixel_points):
    i_under = pixel_points[:,0] < 720
    i_over = pixel_points[:,0] > 0
    j_under = pixel_points[:,1] < 1280
    j_over = pixel_points[:,1] > 0
    pixel_inliers = i_under*i_over*j_under*j_over
    return pixel_inliers


# def pixel2cam(pixel_points, depths, kinv):
#     depth_points = pixel_points * np.array(depths).reshape(-1,1)
#     camera_points = depth_points @ kinv.T
#     return camera_points[:,:-1]


# get indices of 3d points
def select_apple_points(camera_points, pixel_points, inliers, mask):
    apple_points_dict = dict()
    for idx, pt in zip(np.where(inliers)[0], np.floor(pixel_points[inliers,:2])):
        new_val = camera_points[idx,2]
        r,c = np.int32(pt)
        key = f"{r}-{c}"
        if apple_points_dict.get(key, np.inf) > new_val:
            r,c = np.int32(pt)
            if mask[c,r] == 1:
                apple_points_dict[key] = idx
    apple_points_indices = list(apple_points_dict.values())

    return apple_points_indices


# get 3d points
def get_3d_apple_points(points3d, extrinsic, intrinsic, mask):
    cam_pts = get_camera_view(points3d, extrinsic)
    pixel_pts = get_image_view(cam_pts, intrinsic)
    inliers = get_in_view_points(pixel_pts)

    apple_points_indices = select_apple_points(cam_pts, pixel_pts, inliers, mask)

    return apple_points_indices, pixel_pts

In [4]:
class Maps():
    def __init__(self, filenames, mask_root, image_root) -> None:
        self.filenames = filenames
        self.mask_root = mask_root
        self.image_root = image_root

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        mask = np.array(Image.open(os.path.join(self.mask_root, self.filenames[idx])))
        bool_mask = mask > 0
        mask = np.int8(bool_mask)

        image = np.array(Image.open(os.path.join(self.image_root, self.filenames[idx])))

        return mask, image

In [5]:
filenames = [img_dict[idx] for idx in img_ids]
masks_path = "./created_masks/"
images_path = "./images/"
maps = Maps(filenames, masks_path, images_path)

In [6]:
set_apple_points = set()
for i in range(len(maps)):
    mask, image = maps[i]

    extrinsic = get_extrinsic_matrix(poses[i])
    K = get_K_matrix(cam_params)
    apple_points_indices, pixel_pts = get_3d_apple_points(pts3d, extrinsic, K, mask)

    for pt_idx in apple_points_indices:
        set_apple_points.add(pt_idx)
all_apple_points = np.array(list(set_apple_points))

In [7]:
apple_color_points3d = np.concatenate([pts3d[all_apple_points], 255*rgb3d[all_apple_points]], axis=1)
np.save("apple_points3d.npy", apple_color_points3d)